In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import re
pattern = re.compile(r'\(?\b\d+/\d+\b\)?')
def handle_percentage(i):
    if '%' in i:    
        if i.endswith('%)'):
            temp_list=i.split('(')[0].strip().split('/')
        else:
            if len(pattern.findall(i))>0:        
                temp_list=pattern.findall(i)[0].strip('()').split('/')
            else:
                return None
    else:
        temp_list=i.strip().split('/')

    for t in temp_list:
        if not t.isnumeric():
            return None

    if len(temp_list)==2:
        if int(temp_list[0])>int(temp_list[1]):
            temp_list[0]=str(int(int(temp_list[0])*int(temp_list[1])/100))
     
    return temp_list

def extract_marker_statistics(data_df, query, topn=5):
    temp_df=data_df[['tumor type', 'tumor site', query]].dropna()
    try:
        temp_df[query]=temp_df[query].str.lower().str.strip()
    except:
        return {}, []

    stats={}
    valid_temp=temp_df[~temp_df[query].str.contains('na').fillna(True)]
    valid_temp=valid_temp[valid_temp[query].str.contains('/')]
    valid_temp[query]=valid_temp[query].apply(handle_percentage)
    valid_temp.dropna(inplace=True)
    stats['IHC Marker']=query
    stats['# abstracts']=len(temp_df.index.unique())
    stats['# abstracts w/ IHC positive rates']=len(valid_temp.index.unique())
    stats['Total IHC Positives']=valid_temp[query].apply(lambda x: x[0]).astype('float').sum()
    stats['Total Cohort Size']=valid_temp[query].apply(lambda x: x[1]).astype('float').sum()
    stats['Positive Rates']=stats['Total IHC Positives']/stats['Total Cohort Size']
    valid_temp[f'positive']=valid_temp[query].apply(lambda x: x[0]).astype('float')
    valid_temp[f'sample_number']=valid_temp[query].apply(lambda x: x[1]).astype('float')

    merged_valid_temp=valid_temp[['tumor type', 'tumor site', f'positive', f'sample_number']].groupby('tumor type').sum().sort_values(by=f'sample_number', ascending=False)
    merged_valid_temp['positive_rate']=merged_valid_temp[f'positive']/merged_valid_temp[f'sample_number']
    print(merged_valid_temp)
    detailed=merged_valid_temp.sort_values(by=f'sample_number', ascending=False).head(topn).reset_index().to_dict(orient='records')
    result=[]
    for temp_dict in detailed:
        temp_dict['marker']=query
        result.append(temp_dict)
    result=pd.DataFrame(result)[['marker','tumor type', 'positive', 'sample_number', 'positive_rate']]
    return stats, result

In [ ]:
data_df

In [ ]:
data_df=pd.read_csv('inference_umls_mapped_data.tsv',sep='\t')
column_names=[c for c in data_df.columns if c!='tumor type' and c!='tumor site']

In [ ]:
query=column_names[0]
stats, result=extract_marker_statistics(data_df, query, topn=5)

In [ ]:
print(stats)

In [ ]:
result